In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

## Options

In [2]:
# parse options
problem = 'twelve_pieces_process.json' # 'pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = 'results'

recompute_action_states = False
load_external_movements = False

In [3]:
# generic planning options
id_only = None # 'A2_M1' # None | 'A273_M0'

low_res = False
plan_impacted = False

#     'nonlinear',
#     'linear',
#     'id_only', # 'Compute only for movement with a specific tag, e.g. `A54_M0`.'
#     'free_motion_only', # 'Only compute free motions.'
#     'propagate_only', # 'Only do state propagation and impacted movement planning.'
solve_mode = 'linear'

viz_upon_found = False

In [4]:
# client options
viewer = True
verbose = True

debug = False
diagnosis = False

watch = True
step_sim = False

disable_env = False
reinit_tool = False

write = False
save_now = False

In [5]:
from collections import namedtuple
PlanningArguments = namedtuple('PlanningArguments', ['problem', 'viewer', 'debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
                                             'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

args = PlanningArguments(problem, viewer, debug, diagnosis, id_only, solve_mode, viz_upon_found, save_now, write, plan_impacted, watch, step_sim, verbose)

## Parse process from json

In [6]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, get_process_path, save_process

In [7]:
process = parse_process(problem, subdir=problem_subdir)
result_path = get_process_path(problem, subdir='results')
if len(process.movements) == 0:
    cprint('No movements found in process, trigger recompute actions.', 'red')
    recompute_action_states = True
if recompute_action_states:
    cprint('Recomputing Actions and States', 'cyan')
    recompute_action_states(process)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results\twelve_pieces_process.json


In [8]:
from copy import deepcopy
unsolved_process = deepcopy(process)

In [9]:
# # force load external if only planning for the free motions
# load_external_movements = load_external_movements or free_motion_only or id_only is not None
# if load_external_movements:
#     ext_movement_path = os.path.dirname(result_path)
#     cprint('Loading external movements from {}'.format(ext_movement_path), 'cyan')
#     process.load_external_movements(ext_movement_path)
#     if recompute_action_states:
#         save_process(process, result_path)
#         cprint('Recomputed process saved to %s' % result_path, 'green')

In [10]:
# from collections import defaultdict

# # runtime_data = defaultdict(dict)
# with open('runtime_data.json', 'r') as f:
#     runtime_data = json.load(f)

## Start client

In [74]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=False, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [76]:
client.disconnect()

In [11]:
from integral_timber_joints.planning.robot_setup import GANTRY_ARM_GROUP, GANTRY_GROUP, BARE_ARM_GROUP
from compas.robots import Joint
import numpy as np

joint_names = robot.get_configurable_joint_names(group=GANTRY_ARM_GROUP)
joint_types = robot.get_joint_types_by_names(joint_names)
# 0.1 rad = 5.7 deg
joint_jump_threshold = {jt_name : np.pi/6 \
        if jt_type in [Joint.REVOLUTE, Joint.CONTINUOUS] else 0.1 \
        for jt_name, jt_type in zip(joint_names, joint_types)}

options = {
    'distance_threshold' : 0.0012,
    'frame_jump_tolerance' : 0.0012,
    'verbose' : verbose,
    'jump_threshold' : joint_jump_threshold,
#     'max_distance' : args.max_distance,
}

# Plan movements for a beam

In [77]:
from copy import copy, deepcopy
import time

# start_time = time.time()
# process = copy(unsolved_process)
# print('Copy time: {:.5f}'.format(time.time() - start_time))

start_time = time.time()
process = deepcopy(unsolved_process)
print('Deepcopy time: {:.5f}'.format(time.time() - start_time))

Deepcopy time: 4.22508


In [78]:
beam_id = 'b4'
process.get_movement_summary_by_beam_id(beam_id)

=====
Summary:
---
(0) RoboticFreeMovement(#A32_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to get clamp., traj 0) 
priority 0 | has start conf False, TCP True | has end conf False, TCP True | has traj None
---
(1) RoboticLinearMovement(#A32_M1, Linear Advance to Storage Frame of CL3 ('c1'), to get tool., traj 0) 
priority 0 | has start conf False, TCP True | has end conf True, TCP True | has traj None
---
(2) RoboticDigitalOutput(#A32_M2, Toolchanger Lock CL3 ('c1')) 
priority -1 | has start conf True, TCP True | has end conf True, TCP True
---
(3) RoboticDigitalOutput(#A32_M3, CL3 ('c1') Open Gripper to release itself from storage pad.) 
priority -1 | has start conf True, TCP True | has end conf True, TCP True
---
(4) RoboticLinearMovement(#A32_M4, Linear Retract 1 of 2 after getting CL3 ('c1') from storage., traj 0) 
priority 0 | has start conf True, TCP True | has end conf False, TCP True | has traj None
---
(5) RoboticLinearMovement(#A32_M5, Linear Retract 2 of 2 aft

In [80]:
client, robot, _ = load_RFL_world(viewer=False, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [81]:
import json
from collections import defaultdict
from integral_timber_joints.planning.run import compute_movements_for_beam_id, plan_for_beam_id_with_restart
# beam_id = process.get_beam_id_from_movement_id(args.id_only)

beam_id = 'b4'
# options['movement_id_range'] = range(36,41) # 35

options.update({
    'debug' : False,
    'verbose' : True,
    'diagnosis' : False,
    'low_res' : False,
    'movement_planning_reattempts' : 1,
    'solve_timeout' : 1800,
})

# PlanningArguments = namedtuple('PlanningArguments', ['problem', 'viewer', debug', 'diagnosis', 'id_only', 'solve_mode', 'viz_upon_found', 
#                                              'save_now', 'write', 'plan_impacted', 'watch', 'step_sim', 'verbose'])

num_trails = 1
# runtime_data = {}

#'linear'
# for solve_mode_ in ['nonlinear', 'linear']:
solve_mode_ = 'linear'
args = PlanningArguments(problem, False, False, False, id_only, solve_mode_, viz_upon_found, save_now, write, 
                         plan_impacted, False, False, True)
all_movements = process.get_movements_by_beam_id(beam_id)
options['movement_id_range'] = list(reversed(range(0, len(all_movements))))

runtime_data[solve_mode_ + '_backward'] = {}
for attempt_i in range(num_trails):
    process = deepcopy(unsolved_process)
    success, trial_data = plan_for_beam_id_with_restart(client, robot, process, beam_id, args, options=options)
    runtime_data[solve_mode_][attempt_i] = trial_data
    with open('figs/{}_runtime_data.json'.format(beam_id), 'w') as f:
        json.dump(runtime_data, f)

print('Done')
client.disconnect()

##########
linear | Inner Trail #0



* compute movement ids: ['A43_M5', 'A43_M4', 'A43_M3', 'A43_M2', 'A43_M1', 'A43_M0', 'A42_M6', 'A42_M5', 'A42_M4', 'A42_M3', 'A42_M2', 'A42_M1', 'A42_M0', 'A41_M5', 'A41_M4', 'A41_M3', 'A41_M2', 'A41_M1', 'A41_M0', 'A40_M6', 'A40_M5', 'A40_M4', 'A40_M3', 'A40_M2', 'A40_M1', 'A40_M0', 'A39_M4', 'A39_M3', 'A39_M2', 'A39_M1', 'A39_M0', 'A38_M5', 'A38_M4', 'A38_M3', 'A38_M2', 'A38_M1', 'A38_M0', 'A37_M5', 'A37_M4', 'A37_M3', 'A37_M2', 'A37_M1', 'A37_M0', 'A36_M4', 'A36_M3', 'A36_M2', 'A36_M1', 'A36_M0', 'A35_M5', 'A35_M4', 'A35_M3', 'A35_M2', 'A35_M1', 'A35_M0', 'A34_M5', 'A34_M4', 'A34_M3', 'A34_M2', 'A34_M1', 'A34_M0', 'A33_M5', 'A33_M4', 'A33_M3', 'A33_M2', 'A33_M1', 'A33_M0', 'A32_M5', 'A32_M4', 'A32_M3', 'A32_M2', 'A32_M1', 'A32_M0']
----------
(71)
Movement planning attempt 0
RoboticLinearMovement(#A43_M5, Linear Retract from storage after placing CL3 ('c2') in storage, traj 0)
One-sided Cartesian planning : start conf set, forward mode
	cartesia

end conf FK inconsistent (0.00002 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #2
No Cartesian motion found, due to IK plan is not found.!
Append ladder graph fails: no edge built between 0-1
Ladder graph cost: None
No Cartesian motion found, due to !
Cartesian Path planning (w/ prespecified st or end conf) failure after
IterativeIK attempts of 3 + 1 LadderGraph attempt.
No linear movement found for RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 0).
Planning fails! Go back to the command line now!
No plan found for A43_M2 after 1 attempts! RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 0)
No success for linear planning.
##########
linear | Inner Trail #3



* compute movemen

end conf FK inconsistent (0.00002 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #2
Conf disagreement found, FK center point diff 0.00001(m), but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 2 path failure (by IterativeIK) over 3 samples.
~~~~~
	Propagate states for (68) : RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 1)
	$ Impacted (backward): (67) RoboticLinearMovement(#A43_M1, Linear Approach 1 of 2 to place CL3 ('c2') in storage., traj 0)
	- Altered (forward): (69) RoboticDigitalOutput(#A43_M3, Close Gripper to lock CL3 ('c2') onto storage pad.)
	- Altered (forward): (70) RoboticDigitalOutput(#A43_M4, Toolchanger Unlock CL3 ('c2'))
----------
(67)
Movement planning attempt 0
Robot

Free movement found for RoboticFreeMovement(#A43_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 0)!
~~~~~
	Propagate states for (66) : RoboticFreeMovement(#A43_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 1)
	$ Impacted (backward): (65) RoboticLinearMovement(#A42_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c2') from structure., traj 0)
----------
(65)
Movement planning attempt 0
RoboticLinearMovement(#A42_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c2') from structure., traj 0)
end conf FK inconsistent (0.00003 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is found but collision violated.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #2
No Cartesian motion found, due to IK plan is not found.!
Ladder

end conf FK inconsistent (0.00002 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
Conf disagreement found, FK center point diff 0.00001(m), but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.
~~~~~
	Propagate states for (68) : RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 1)
	$ Impacted (backward): (67) RoboticLinearMovement(#A43_M1, Linear Approach 1 of 2 to place CL3 ('c2') in storage., traj 0)
	- Altered (forward): (69) RoboticDigitalOutput(#A43_M3, Close Gripper to lock CL3 ('c2') onto storage pad.)
	- Altered (forward): (70) RoboticDigitalOutput(#A43_M4, Toolchanger Unlock CL3 ('c2'))
----------
(67)
Movement planning attempt 0
RoboticLinearMovement(#A43_M1, Linear Approach 1 of 2 to place CL3 ('c2') in storage., traj 0)
end conf FK inconsistent (0.00003 m) with given current frame in

...

Free movement found for RoboticFreeMovement(#A41_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to place clamp in storage., traj 0)!
~~~~~
	Propagate states for (53) : RoboticFreeMovement(#A41_M0, Free Move reach Storage Approach Frame of CL3 ('c1'), to place clamp in storage., traj 1)
	$ Impacted (backward): (52) RoboticLinearMovement(#A40_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c1') from structure., traj 0)
----------
(52)
Movement planning attempt 0
RoboticLinearMovement(#A40_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c1') from structure., traj 0)
end conf FK inconsistent (0.00003 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is found but collision violated.!
	cartesian trial #2
No Cartesian motion found, due to IK plan is not found.!
Ladder

end conf FK inconsistent (0.00002 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #2
No Cartesian motion found, due to IK plan is not found.!
Append ladder graph fails: no edge built between 0-1
Ladder graph cost: None
No Cartesian motion found, due to !
Cartesian Path planning (w/ prespecified st or end conf) failure after
IterativeIK attempts of 3 + 1 LadderGraph attempt.
No linear movement found for RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 0).
Planning fails! Go back to the command line now!
No plan found for A43_M2 after 1 attempts! RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 0)
No success for linear planning.
##########
linear | Inner Trail #86



* compute moveme

end conf FK inconsistent (0.00002 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
Conf disagreement found, FK center point diff 0.00001(m), but still returns a trajectory. Please be cautious.
Plan found by IterativeIK! After 1 path failure (by IterativeIK) over 2 samples.
~~~~~
	Propagate states for (68) : RoboticLinearMovement(#A43_M2, Linear Approach 2 of 2 to place CL3 ('c2') in storage., traj 1)
	$ Impacted (backward): (67) RoboticLinearMovement(#A43_M1, Linear Approach 1 of 2 to place CL3 ('c2') in storage., traj 0)
	- Altered (forward): (69) RoboticDigitalOutput(#A43_M3, Close Gripper to lock CL3 ('c2') onto storage pad.)
	- Altered (forward): (70) RoboticDigitalOutput(#A43_M4, Toolchanger Unlock CL3 ('c2'))
----------
(67)
Movement planning attempt 0
RoboticLinearMovement(#A43_M1, Linear Approach 1 of 2 to place CL3 ('c2') in stora

Free movement found for RoboticFreeMovement(#A43_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 0)!
~~~~~
	Propagate states for (66) : RoboticFreeMovement(#A43_M0, Free Move reach Storage Approach Frame of CL3 ('c2'), to place clamp in storage., traj 1)
	$ Impacted (backward): (65) RoboticLinearMovement(#A42_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c2') from structure., traj 0)
----------
(65)
Movement planning attempt 0
RoboticLinearMovement(#A42_M6, Linear Retract 2 of 2 from storage after picking up CL3 ('c2') from structure., traj 0)
end conf FK inconsistent (0.00003 m) with given current frame in end state.
One-sided Cartesian planning : end conf set, backward mode
	cartesian trial #0
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #1
No Cartesian motion found, due to IK plan is not found.!
	cartesian trial #2
No Cartesian motion found, due to IK plan is not found.!
Ladder graph cost: None
N

KeyboardInterrupt: 

In [58]:
with open('figs/{}_runtime_data.json'.format(beam_id), 'w') as f:
    json.dump(runtime_data, f)

In [68]:
for solve_mode_ in ['linear', 'nonlinear']:
    print('='*20)
    for i, tdata in runtime_data[solve_mode_].items():
        print('#{}-T#{}: {}'.format(solve_mode_, i, [d['success'] for di, d in tdata.items()]))
        sc = any([d['success'] for di, d in tdata.items()])
        
        total_runtime = []
        for i, trial_data in tdata.items():
            trial_profiles = trial_data['profiles']
            runtime_per_move = [sum(trial_profiles[mid]['plan_time']) for mid in trial_profiles]
            total_runtime.append(sum(runtime_per_move))
      
        cprint('{} - BT {} | time {:.2f}'.format(sc, len(tdata), sum(total_runtime)), 'green' if sc else 'red')
        print('---')

#linear-T#0: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
False - BT 38 | time 1697.92
---
#linear-T#1: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
False - BT 42 | time 1629.65
---
#linear-T#2: [False, False, False, False, False, False, False, False, False, True]
True - BT 10 | time 475.35
---
#linear-T#3: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, F

# Diagram

In [63]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

solve_mode_ = 'linear'

# total_rows = 0
# for i, d in runtime_data[solve_mode_].items():
#     total_rows += len(d)+1

for attempt_i, s_rdata in runtime_data[solve_mode_].items():
    fig = make_subplots(rows=len(s_rdata)+1, cols=2)
    total_runtime = []
    success = any([d['success'] for di, d in s_rdata.items()])
    for i, trial_data in s_rdata.items():
        trial_profiles = trial_data['profiles']
        mid_keys = sorted(map(int, trial_profiles.keys()))
        runtime_per_move = [sum(trial_profiles[mid]['plan_time'])/len(trial_profiles[mid]['plan_time']) for mid in mid_keys]
        success_colors = ['#99C24D' if any(trial_profiles[mid]['plan_success']) else '#F18F01' for mid in mid_keys]
        total_runtime.append(sum(runtime_per_move))

        fig.append_trace(go.Scatter(x=mid_keys,
                                y=runtime_per_move,
                                mode='markers',
                                marker_color=success_colors,
                                text=[process.get_movement_by_movement_id(trial_profiles[mid]['movement_id'][0]).short_summary \
                                      for mid in mid_keys], # hover text goes here
                                name='#{}-feasibility'.format(i),),
                      row=int(i)+1, col=1
                        )

        fig.append_trace(go.Scatter(x=mid_keys,
                                        y=runtime_per_move,
                                        mode='markers',
                                            marker=dict(
                                            size=5,
                                            color=[trial_profiles[mid]['sample_order'][0] for mid in mid_keys], #set color equal to a variable
                                            colorscale='Viridis', # one of plotly colorscales
                                            showscale=True
                                        ),
                                        text=['S#{}-{}'.format(trial_profiles[mid]['sample_order'][0], process.get_movement_by_movement_id(trial_profiles[mid]['movement_id'][0]).short_summary) \
                                              for mid in mid_keys], # hover text goes here
                                        name='#{}-sample order'.format(i),),
                      row=int(i)+1, col=2
                        )

    fig.append_trace(go.Scatter(x=list(range(len(s_rdata))),y=total_runtime), row=len(s_rdata)+1, col=1)    
    fig.update_layout(title='{} planning: trail #{}'.format(solve_mode_, attempt_i))
    fig.write_html("figs/{}-{}-trail_{}_success-{}_BT-{}_time-{:.1f}.html".format(beam_id, solve_mode_, attempt_i, success, len(s_rdata), sum(total_runtime)))
# fig.show()

# Save runtime data

In [106]:
runtime_data.keys()

dict_keys(['linear', 'nonlinear'])

In [18]:
from integral_timber_joints.planning.parsing import save_process_and_movements

beam_all_movements = process.get_movements_by_beam_id(beam_id)
if 'movement_id_range' not in options:
    altered_movements = beam_all_movements
else:
    altered_movements = [beam_all_movements[mid] for mid in options['movement_id_range']]
save_process_and_movements(problem, process, altered_movements, save_temp=True)

KeyError: 'movement_id_range'

# Disconnect client

In [48]:
client.disconnect()

# Plan only one movement

In [29]:
# if id_only:
#     beam_id = process.get_beam_id_from_movement_id(id_only)
#     process.get_movement_summary_by_beam_id(beam_id)

In [39]:
from integral_timber_joints.planning.stream import compute_free_movement, compute_linear_movement
from integral_timber_joints.planning.solve import compute_movement

chosen_m = process.get_movement_by_movement_id(id_only)
compute_movement(client, robot, process, chosen_m, options=lm_options, diagnosis=diagnosis)

RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)
end conf FK inconsistent (0.00005 m) with given current frame in end state.
Both start/end confs are pre-specified, problem might be too stiff to be solved.
One-sided Cartesian planning : start conf set, forward mode
	cartesian trial #0
Plan found by IterativeIK! After 0 path failure (by IterativeIK) over 1 samples.


True

In [40]:
from integral_timber_joints.planning.visualization import visualize_movement_trajectory

with pp.WorldSaver():
    visualize_movement_trajectory(client, robot, process, chosen_m, step_sim=True)

===
Viz:
RoboticLinearMovement(#A2_M1, Linear Advance to Final Frame of Beam ('b0'), traj 1)


Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
Step conf. 
End state. 
